In [ ]:
import jax
import jax.numpy as jnp
import orbax
import pathlib
from xlron.environments.rsa import make_rsa_env

In [ ]:
# Load model and visualize it using penzai
config_dict = {}
env, env_params = make_rsa_env(config_dict)
rng = jax.random.PRNGKey(0)
obsv, env_state = env.reset(rng, env_params)
obsv = (env_state.env_state, env_params) if config_dict.get("USE_GNN") else tuple([obsv])

# INIT NETWORK
network = ActorCriticGNN(
                activation=config.ACTIVATION,
                num_layers=config.NUM_LAYERS,
                num_units=config.NUM_UNITS,
                gnn_latent=config.gnn_latent,
                message_passing_steps=config.message_passing_steps,
                # output_edges_size must equal number of slot actions
                output_edges_size=math.ceil(env_params.link_resources / env_params.aggregate_slots),
                output_nodes_size=config.output_nodes_size,
                output_globals_size=config.output_globals_size,
                gnn_mlp_layers=config.gnn_mlp_layers,
                normalise_by_link_length=config.normalize_by_link_length,
                mlp_layer_norm=config.LAYER_NORM,
            )
init_x = (env_state.env_state, env_params)
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
model = orbax_checkpointer.restore(pathlib.Path(FLAGS.MODEL_PATH))
network_params = model["model"]["params"]